In [1]:
"""
Format dataset, we read the file for the desired subject, and parse the data to extract:
- samplingRate
- trialLength
- X, a M x N x K matrix, which stands for trial x chan x samples
                         the actual values are 160 x 15 x 2560
- y, a M vector containing the labels {0,1}

ref:
Dataset description: https://lampx.tugraz.at/~bci/database/002-2014/description.pdf
"""
import scipy.io as sio
import numpy as np
from os import listdir
from os.path import isfile, join

# prepare data containers
y = []
X = []

folder = "../BBCIData"

for f in listdir(folder):
    # read file
    d1T = sio.loadmat(folder + "/" + f)
    
    samplingRate = d1T['data'][0][0][0][0][3][0][0]
    trialLength = 7*samplingRate


    # run through all training runs
    for run in range(len(d1T['data'][0])):
        y.append(d1T['data'][0][run][0][0][2][0]) # labels
        timestamps = d1T['data'][0][run][0][0][1][0] # timestamps
        rawData = d1T['data'][0][run][0][0][0].transpose() # chan x data

        # parse out data based on timestamps
        for start in timestamps:
            end = start + trialLength
            X.append(rawData[:,start:end]) #15 x 2560

    del rawData
    del d1T

# arrange data into num7py arrays
# also torch expect float32 for samples
# and int64 for labels {0,1}
X = np.array(X).astype(np.float32)
y = (np.array(y).flatten()-1).astype(np.int64)
print(X.shape)
print(y.shape)



(2240, 15, 3584)
(2240,)


In [2]:

import numpy as np


from brainDecodeMOABBWrapper.SkLearnBrainDecode import SkLearnBrainDecode

#from pyriemann.estimation import XdawnCovariances
#from pyriemann.tangentspace import TangentSpace
#from pyriemann.utils.viz import plot_confusion_matrix
#from pyriemann.estimation import Covariances
#from pyriemann.classification import TSclassifier, MDM

from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from random import randint
from matplotlib import pyplot as plt

###############################################################################
# Decoding in tangent space with a logistic regression

n_components = 2  # pick some components

labels = y
epochs_data = X


# Define a monte-carlo cross-validation generator (reduce variance):
cv = KFold(len(labels), 10, shuffle=True, random_state=randint(1,5000))

print("epoch data:")
print(epochs_data.shape)




classifer = SkLearnBrainDecode()


preds = np.zeros(len(labels))

print("labels:")
print(labels.shape)

for train_idx, test_idx in cv:
    y_train, y_test = labels[train_idx], labels[test_idx]

    classifer.fit(epochs_data[train_idx], y_train)
    #preds[test_idx] = clf.predict(epochs_data[test_idx])

# Printing the results
#acc = np.mean(preds == labels)
#print("Classification accuracy: %f " % (acc))

#names = ['audio left', 'audio right', 'vis left', 'vis right']
#plot_confusion_matrix(preds, labels, names)
#plt.show()

/home/fred/anaconda2/envs/Torchpy35/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


epoch data:
(2240, 15, 3584)
labels:
(2240,)
Epoch 0
Train  Loss: 2.14122
Train  Accuracy: 49.4%
Test   Loss: 2.14673
Test   Accuracy: 51.2%
Epoch 1


KeyboardInterrupt: 